In [20]:
import os

from dotenv import load_dotenv 
from langchain_openai import ChatOpenAI,OpenAIEmbeddings

load_dotenv()

os.environ["OPEN_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["COHERE_API_KEY"] = os.getenv('COHERE_API_KEY')

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")



In [21]:
# Create vector store

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores.faiss import FAISS

index = faiss.IndexFlatL2(len(OpenAIEmbeddings().embed_query(" ")))


vectorstore = FAISS(
  embedding_function=OpenAIEmbeddings(),
  index=index,
  docstore=InMemoryDocstore(),
  index_to_docstore_id={}
)



In [28]:

from langchain_cohere import CohereEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma


# load embeddings
embeddings_model = CohereEmbeddings(model="embed-english-v3.0")


# Docs to index
urls = [
    "https://www.deeplearning.ai/the-batch/how-agents-can-improve-llm-performance/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-2-reflection/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-3-tool-use/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-4-planning/?ref=dl-staging-website.ghost.io",
    "https://www.deeplearning.ai/the-batch/agentic-design-patterns-part-5-multi-agent-collaboration/?ref=dl-staging-website.ghost.io"
]


# Load
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

vectorstore = Chroma.from_documents(
  documents=doc_splits,
  collection_name='rag',
  embedding=embeddings_model,
)


  


TypeError: VectorStore.add_documents() missing 1 required positional argument: 'self'